In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from split_llm.glm6b.wrapped_layer import Attention_GLM_Wrapped, copy_attention, FeedForward_GLM_Wrapped, copy_feedforward
from split_llm.glm6b.utils import generate_position_ids

/home/zf/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llm_bases.chatglm6b import ChatGML6B
glm = ChatGML6B()

Loading checkpoint shards: 100%|██████████| 8/8 [00:12<00:00,  1.50s/it]


In [3]:
transformer_layer = glm.condgen.transformer.layers[0].float()
attn_wrapped = Attention_GLM_Wrapped(4096, 32, 0)
ff_wrapped = FeedForward_GLM_Wrapped(4096, 32, 0)
copy_attention(transformer_layer, attn_wrapped)
copy_feedforward(transformer_layer, None, ff_wrapped)
attn_wrapped.cuda()
ff_wrapped.cuda()

FeedForward_GLM_Wrapped(
  (layernorm_in): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  (mlp_dense_in): Linear(in_features=4096, out_features=16384, bias=True)
  (mlp_dense_out): Linear(in_features=16384, out_features=4096, bias=True)
  (layernorm_out): Identity()
)

In [4]:
import torch
xs = torch.normal(0, 1, [10, 1, 4096]).cuda()
position_ids = generate_position_ids(10, 10).cuda()

In [5]:
# Generate the hidden representations in a normal transformer layer
attn_out_raw = attn_wrapped(xs, position_ids)
attn_out = attn_out_raw + (2 * 28) ** 0.5 * xs
output_wrapped = ff_wrapped(attn_out)

In [6]:
try:
    import sys
    del sys.modules['split_llm.glm6b.utils']
    del sys.modules['split_llm.protocols.base_protocols']
    del sys.modules['split_llm.protocols.ss_mul_with_memory']
    del sys.modules['split_llm.protocols.element_wise']
    del sys.modules['split_llm.glm6b.secure_inference']
    del sys.modules['split_llm.glm6b.wrapped_layer']
    del sys.modules['split_llm.common.torch_utils']
    print("delete complete!")
except Exception as e:
    print(e)
from split_llm.glm6b.secure_inference import GLM_AttentionProtocol, GLM_FeedForwardProtocol_PlainWeights
from split_llm.common.torch_utils import relative_error

'split_llm.protocols.base_protocols'


In [7]:
from split_llm.common.communication import Communication, Node, SimulatedCommunication
communication = SimulatedCommunication(["n0", "n1", "n2"])
communication.new_stage("Test")

n0 = Node(communication, "n0")
n1 = Node(communication, "n1")
n2 = Node(communication, "n2")

In [8]:
n0.space.attentions = [attn_wrapped.cuda()]
n0.space.ffs = n1.space.ffs = [ff_wrapped.cuda()]

In [9]:
attn_protocol = GLM_AttentionProtocol(
    n0, n1, n2, 0, 10, device="cuda"
)
attn_protocol.prepare()
attn_protocol.offline_execute(10)

ff_protocol = GLM_FeedForwardProtocol_PlainWeights(
    n0, n1, n2, 0, 10, device="cuda"
)
ff_protocol.prepare()
ff_protocol.offline_execute(10)

In [10]:
x0 = torch.normal(0, 1, [10, 1, 4096]).cuda()
x1 = xs.cuda() - x0
n0.storage[f"{attn_protocol.name}:x0"] = x0
n1.storage[f"{attn_protocol.name}:x1"] = x1

In [11]:
# Test the attention layer protocols
attn_protocol.online_execute()
attn_out_shared = n0.storage[f"{attn_protocol.name}:z0"] + n1.storage[f"{attn_protocol.name}:z1"]
print(f"attn_out error: {relative_error(attn_out_shared, attn_out_raw):.5f}")

attn_out error: 0.00075


In [12]:
n0.storage[f"{ff_protocol.name}:x0"] = n0.storage[f"{attn_protocol.name}:z0"] + (2 * 28) ** 0.5 * n0.storage[f"{attn_protocol.name}:x0"]
n1.storage[f"{ff_protocol.name}:x1"] = n1.storage[f"{attn_protocol.name}:z1"] + (2 * 28) ** 0.5 * n1.storage[f"{attn_protocol.name}:x1"]

In [13]:
ff_protocol.online_execute()
ff_out_shared = n0.storage[f"{ff_protocol.name}:z0"] + n1.storage[f"{ff_protocol.name}:z1"]
print(f"ff_out error: {relative_error(ff_out_shared, output_wrapped):.5f}")

ff_out error: 0.00004
